In [3]:
!pip install tqdm

Could not fetch URL https://pypi.org/simple/tqdm/: There was a problem confirming the ssl certificate: HTTPSConnectionPool(host='pypi.org', port=443): Max retries exceeded with url: /simple/tqdm/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self signed certificate in certificate chain (_ssl.c:997)'))) - skipping
Could not fetch URL https://pypi.org/simple/pip/: There was a problem confirming the ssl certificate: HTTPSConnectionPool(host='pypi.org', port=443): Max retries exceeded with url: /simple/pip/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self signed certificate in certificate chain (_ssl.c:997)'))) - skipping


ERROR: Could not find a version that satisfies the requirement tqdm (from versions: none)
ERROR: No matching distribution found for tqdm


In [4]:

import re
import itertools
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, cross_validate
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import make_scorer, mean_absolute_error, mean_squared_error, r2_score
from sklearn.inspection import permutation_importance
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from joblib import Parallel, delayed
import tensorflow as tf
from tensorflow.keras import metrics
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Input, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense


ModuleNotFoundError: No module named 'tqdm'

In [14]:
dataframe_final = pd.read_parquet('dados_webscraping/dataframe_imoveis_aluguel_pronto_para_predicao.parquet')

print(dataframe_final.shape)
display(dataframe_final.head(2))
print("Porcentagem de informações faltantes: ")
display(round(dataframe_final.isna().sum()/dataframe_final.shape[0] * 100, 2).astype(str) + " %")


(518, 29)


,url,endereco,preco,area,quartos,vagas_de_carro,valor_condominio,iptu,mobiliado,piscina,condominio,elevador,jardim,quadra_esportiva,academia,finalidade,tipo,localizacao,latitude,longitude,geometry,nome,distancia_metro,distancia_escola,distancia_unidade_saude,indic_rend,indic_lixo,indic_esgo,indic_agua
0,https://www.zapimoveis.com.br/imovel/aluguel-a...,"Avenida Epitácio Pessoa, 4344 - Lagoa, Rio de ...",12500.0,137.0,3.0,2,1982.0,470.0,False,False,False,False,True,False,False,residencial,apartamento,"Avenida Epitácio Pessoa, Lagoa, Rio de Janeiro...",-22.978197,-43.199374,b'\x01\x01\x00\x00\x00\x97\xef\xd4\x19\x85\x99...,Lagoa,948.063010,437.351027,767.352721,20.550754,99.987981,99.915865,99.987981
2,https://www.zapimoveis.com.br/imovel/aluguel-a...,"Rua Pinto Teles, 660 - Praça Seca, Rio de Jane...",1100.0,60.0,2.0,1,409.0,162.0,False,False,False,False,False,False,False,residencial,apartamento,"Rua Pinto Teles, Praça Seca, Rio de Janeiro, R...",-22.889191,-43.345604,b'\x01\x01\x00\x00\x00\\1(\xc2<\xacE\xc0{\xa2\...,Praça Seca,8011.304617,133.018579,182.882683,3.681449,99.660238,94.377183,98.861081


Porcentagem de informações faltantes: 


url                        0.0 %
endereco                   0.0 %
preco                      0.0 %
area                       0.0 %
quartos                    0.0 %
vagas_de_carro             0.0 %
valor_condominio           0.0 %
iptu                       0.0 %
mobiliado                  0.0 %
piscina                    0.0 %
condominio                 0.0 %
elevador                   0.0 %
jardim                     0.0 %
quadra_esportiva           0.0 %
academia                   0.0 %
finalidade                 0.0 %
tipo                       0.0 %
localizacao                0.0 %
latitude                   0.0 %
longitude                  0.0 %
geometry                   0.0 %
nome                       0.0 %
distancia_metro            0.0 %
distancia_escola           0.0 %
distancia_unidade_saude    0.0 %
indic_rend                 0.0 %
indic_lixo                 0.0 %
indic_esgo                 0.0 %
indic_agua                 0.0 %
dtype: object

In [15]:
colunas_analise = ['preco', 'area', 'quartos', 'vagas_de_carro', 'valor_condominio',
                   'iptu', "mobiliado", "tipo", "distancia_metro", 'distancia_escola', 'distancia_unidade_saude',
                   'indic_rend', 'indic_lixo', 'indic_esgo', 'indic_agua']

display(dataframe_final[colunas_analise].info())

X = dataframe_final[colunas_analise].drop('preco', axis=1)  # Remover a coluna 'preco' de X
y = dataframe_final['preco']

<class 'pandas.core.frame.DataFrame'>
Index: 518 entries, 0 to 1606
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   preco                    518 non-null    float64
 1   area                     518 non-null    float64
 2   quartos                  518 non-null    float64
 3   vagas_de_carro           518 non-null    int64  
 4   valor_condominio         518 non-null    float64
 5   iptu                     518 non-null    float64
 6   mobiliado                518 non-null    bool   
 7   tipo                     518 non-null    object 
 8   distancia_metro          518 non-null    float64
 9   distancia_escola         518 non-null    float64
 10  distancia_unidade_saude  518 non-null    float64
 11  indic_rend               518 non-null    float64
 12  indic_lixo               518 non-null    float64
 13  indic_esgo               518 non-null    float64
 14  indic_agua               518 n

None

In [16]:
# Separar colunas numéricas, categóricas e booleanas
colunas_numericas = X.select_dtypes(include=[np.number]).columns.tolist()
colunas_categoricas = X.select_dtypes(include=[object]).columns.tolist()
colunas_booleanas = X.select_dtypes(include=[bool]).columns.tolist()

# Criar o pré-processador com StandardScaler para colunas numéricas e OneHotEncoder para colunas categóricas e booleanas
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), colunas_numericas),
        ('cat', OneHotEncoder(handle_unknown='ignore'), colunas_categoricas + colunas_booleanas)
    ])

# Aplicar o pré-processador nos dados de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_scaled = preprocessor.fit_transform(X_train)
X_test_scaled = preprocessor.transform(X_test)

# Converter `y` para numpy array (necessário para Keras)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [17]:
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=preprocessor.get_feature_names_out())
pd.set_option('display.max_columns', None)
print(X_train_scaled_df.shape)
display(X_train_scaled_df.head(2))

(414, 16)


,num__area,num__quartos,num__vagas_de_carro,num__valor_condominio,num__iptu,num__distancia_metro,num__distancia_escola,num__distancia_unidade_saude,num__indic_rend,num__indic_lixo,num__indic_esgo,num__indic_agua,cat__tipo_apartamento,cat__tipo_casa,cat__mobiliado_False,cat__mobiliado_True
0,0.235705,0.549450,-0.031817,0.504608,-0.162708,-0.666419,-0.110196,-0.234587,-0.093234,0.609032,0.766037,0.728210,1.0,0.0,1.0,0.0
1,0.041474,-0.538935,-0.031817,-0.978665,-0.681600,-0.508302,-0.266945,-0.430312,-1.092935,0.254257,0.694830,0.619705,0.0,1.0,1.0,0.0


In [18]:
# Função para criar e treinar o modelo de rede neural
def criar_rede_neural(input_shape, learning_rate=0.001, hidden_layers=2, neurons=64, dropout_rate=0.2, activation='relu', 
                      output_activation='linear', optimizer='adam', l2_reg=0.01):
    
    # Verificar parâmetros
    if hidden_layers < 1:
        raise ValueError("O número de camadas ocultas deve ser pelo menos 1.")
    if neurons < 1:
        raise ValueError("O número de neurônios deve ser pelo menos 1.")
    if not 0 <= dropout_rate < 1:
        raise ValueError("A taxa de dropout deve estar entre 0 e 1.")

    model = Sequential()
    model.add(Input(shape=(input_shape,)))
    model.add(Dense(neurons, activation=activation, kernel_regularizer=l2(l2_reg)))

    # Adicionar camadas ocultas
    for _ in range(hidden_layers - 1):
        model.add(Dense(neurons, activation=activation, kernel_regularizer=l2(l2_reg)))
        model.add(Dropout(dropout_rate))

    # Camada de saída (1 neurônio para regressão)
    model.add(Dense(1, activation='linear'))

    # Escolher otimizador
    if optimizer.lower() == 'adam':
        optimizer = Adam(learning_rate=learning_rate)
    elif optimizer.lower() == 'sgd':
        optimizer = SGD(learning_rate=learning_rate)
    else:
        raise ValueError("Otimizador não suportado. Use 'adam' ou 'sgd'.")

    # Compilar o modelo
    model.compile(
        optimizer=optimizer, 
        loss='mse', 
        metrics=['mae'])
    
    return model

def testar_modelo(lr, hidden_layers, neurons, activation, dropout, X_train, y_train, X_test, y_test):

    def learning_rate_scheduler(epoch, lr):
        return float(lr * tf.math.exp(-0.09))

    # Criar o modelo
    model = criar_rede_neural(
        input_shape=X_train.shape[1], 
        learning_rate=lr, 
        hidden_layers=hidden_layers, 
        neurons=neurons, 
        dropout_rate=dropout, 
        activation=activation
    )

    # Treinar o modelo
    model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=0, callbacks=[tf.keras.callbacks.LearningRateScheduler(learning_rate_scheduler)])
    
    # Fazer previsões
    y_pred = model.predict(X_test)
    
    # Calcular métricas
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100  # MAPE
    
    # Retornar resultados e o modelo
    return mae, rmse, r2, mape, model

def testar_varios_modelos_paralelo(X_train, X_test, y_train, y_test):
    
    # Parâmetros para testar
    learning_rates = [0.001, 0.01, 0.1]
    hidden_layer_options = [1, 2, 3]
    neurons_options = [32, 64, 128]
    activation_functions = ['relu', 'tanh', 'sigmoid']
    dropout_rates = [0.2, 0.3, 0.5]
    
    param_combinations = list(itertools.product(learning_rates, hidden_layer_options, neurons_options, activation_functions, dropout_rates))
        
    # Executar os testes em paralelo
    results = Parallel(n_jobs=-1)(
        delayed(testar_modelo)(lr, hidden_layers, neurons, activation, dropout, X_train, y_train, X_test, y_test)
        for lr, hidden_layers, neurons, activation, dropout in tqdm(param_combinations, desc="Testando combinações")
    )
    
    # Identificar o melhor modelo
    best_score = float('inf')
    best_model = None
    
    for mae, rmse, r2, mape, model in results:
        print(f"MAE: {mae:.4f} | RMSE: {rmse:.4f} | R²: {r2:.4f} | MAPE: {mape:.2f}%")
        if mae < best_score:
            best_score = mae
            best_model = model
    
    return best_model, best_score, results


best_model, best_score, results = testar_varios_modelos_paralelo(X_train_scaled, X_test_scaled, y_train, y_test)
best_model = load_model(f'models/melhor_modelo_aluguel_2.keras')

In [19]:
# Treinamento com Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = best_model.fit(
    X_train_scaled, y_train,
    validation_data=(X_test_scaled, y_test),
    epochs=100,
    batch_size=32,
    callbacks=[early_stopping],
    verbose=1
)

# Previsão dos valores de teste
y_pred = best_model.predict(X_test_scaled).flatten()#

Epoch 1/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - loss: 2723954.0000 - mae: 1017.7076 - val_loss: 5329673.0000 - val_mae: 1403.5376
Epoch 2/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 4189061.2500 - mae: 1194.1843 - val_loss: 5329648.5000 - val_mae: 1403.6028
Epoch 3/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 3000559.5000 - mae: 1108.5912 - val_loss: 5330880.0000 - val_mae: 1403.6881
Epoch 4/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 2611310.5000 - mae: 1079.1671 - val_loss: 5330773.5000 - val_mae: 1403.5959
Epoch 5/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 2905488.5000 - mae: 1072.1152 - val_loss: 5329894.0000 - val_mae: 1403.5753
Epoch 6/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 3020534.0000 - mae: 1108.7520 - val_loss: 5330583.5000 - val_mae: 1403.4614
Epoch 7/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 2706760.7500 - mae: 1041.8864 - val_loss: 5330459.0000 - val_mae: 1403.4904
Epoch 8/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/s

In [20]:
# Calculando as métricas
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

# Exibindo as métricas de forma mais bonita
print(f"{'Métricas de Avaliação':^40}")
print(f"{'-'*40}")
print(f"{'MAE':<10}: {mae:.4f}")
print(f"{'RMSE':<10}: {rmse:.4f}")
print(f"{'R²':<10}: {r2:.4f}")
print(f"{'-'*40}")

#         Métricas de Avaliação          
#----------------------------------------
# MAE       : 1354.7818
# RMSE      : 2117.7866
# R²        : 0.8369
#----------------------------------------

#          Métricas de Avaliação          
# ----------------------------------------
# MAE       : 1403.3598
# RMSE      : 2308.7338
# R²        : 0.8061
# ----------------------------------------

         Métricas de Avaliação          
----------------------------------------
MAE       : 1403.6027
RMSE      : 2308.6031
R²        : 0.8061
----------------------------------------
